# The `QuantumState` class

Express an N-qubit quantum state as an array of N columns, where rows are binary bit values, complemented by a vector of coefficients. Note the similarity with the symplectic representation - indeed, the QuantumState class will infact set the above array as the X block in a PauliwordOp, with the Z block its complement.

What we are doing here is writing $|0\rangle = Z|0\rangle$ and $|1\rangle = X|0\rangle$, which ensures correct phases when multiplying the state by Pauli operators, since

$$X|0\rangle = XZ|0\rangle = -iY|0\rangle = |1\rangle,\; X|1\rangle = XX|0\rangle = |0\rangle$$
$$Y|0\rangle = YZ|0\rangle = iX|0\rangle = i|1\rangle,\; Y|1\rangle = YX|0\rangle = -iZ|0\rangle = -i|0\rangle$$
$$Z|0\rangle = ZZ|0\rangle = |0\rangle,\; Z|1\rangle = ZX|0\rangle = iY|0\rangle = -|1\rangle$$

Finally, we have $$| \vec{b} \rangle = \bigotimes_{i=1}^N \big(b_i X + (1-b_i) Z\big) | \vec{0} \rangle$$ and we may drop the zero vector and use the functionality of PauliwordOp to manipulate quantum states. In this represenation, a quantum state is stored as an operator consisting of Paulis $X, Z$, which is implicitly applied to the zero (or vacuum) state. 

In [1]:
import numpy as np
from symred.utils import unit_n_sphere_cartesian_coords
from symred.symplectic_form import PauliwordOp, QuantumState, array_to_QuantumState, AnsatzOp, ObservableOp
from functools import cached_property
from typing import Union

Firstly, note the correct phases under multiplication by a Pauli Y:

In [2]:
zero = QuantumState([[0]])
one = QuantumState([[1]])
Y = PauliwordOp(['Y'], [1])

print(f'{zero} -> {Y * zero}')
print(f'{one} -> {Y * one}')

 1.0000000000 |0> ->  0.0000000000+1.0000000000j |1>
 1.0000000000 |1> ->  0.0000000000-1.0000000000j |0>


Now, let's see what happens when we apply the Hadamard gate to the zero state:

In [3]:
psi = QuantumState([[0,0]])
had = PauliwordOp(['XZ','ZX','XX', 'ZZ'], np.ones(4)/2) # 2-qubit Hadamard gate decomposed over Paulis
eq_superposition = had * psi
print(f'Zero state: {psi}\n')
print(f'After application of the Hadamard gate:\n\n{eq_superposition}')

Zero state:  1.0000000000 |00>

After application of the Hadamard gate:

 0.5000000000+0.0000000000j |00> +
 0.5000000000+0.0000000000j |01> +
 0.5000000000+0.0000000000j |10> +
 0.5000000000-0.0000000000j |11>


Observe that the QuantumState is represented internally by its `state_op`, a PauliwordOp object that governs its behaviour under multiplication

In [4]:
print(eq_superposition.state_op)

0.5000000000+0.0000000000j ZZ +
0.5000000000+0.0000000000j ZX +
0.5000000000+0.0000000000j XZ +
0.5000000000-0.0000000000j XX


Try evaluating expectation values for randomly generated state and Hermitian operators:

In [5]:
def random_state(num_qubits, num_terms):
    """ Generates a random normalized QuantumState
    """
    # random binary array with N columns, M rows
    random_state = np.random.randint(0,2,(num_terms,num_qubits))
    # random vector of coefficients
    coeff_vec = (
        np.random.rand(num_terms) + 
        np.random.rand(num_terms)*1j
    )
    return QuantumState(random_state, coeff_vec).normalize

def random_herm_op(num_qubits, num_terms):
    """ Generates a random PaulwordOp
    """
    # random binary array with 2N columns, M rows
    random_symp_mat = np.random.randint(0,2,(num_terms,num_qubits*2))
    # random vector of coefficients
    coeff_vec = np.random.rand(num_terms)
    coeff_vec/=coeff_vec[::-1]
    coeff_vec*=(2*np.random.randint(0,2,num_terms)-1)
    return PauliwordOp(random_symp_mat, coeff_vec)

In [6]:
N = 10 # number of qubits
M = 10 # number of terms

psi_1 = random_state(N, M)
psi_2 = random_state(N, M)

print(psi_1)
print()
print(psi_2)

 0.2857951443+0.2439316419j |1001011011> +
 0.0961499240+0.1741052917j |0001110010> +
 0.4072587938+0.2188506746j |0101101011> +
 0.1615444965+0.0841882783j |1100011101> +
 0.0357123616+0.0978641215j |1110101111> +
 0.2376001625+0.3756436063j |0011111011> +
 0.0836150445+0.3307325522j |0000001010> +
 0.0364342812+0.0307412645j |0110101000> +
 0.0347807077+0.2929500845j |1010100011> +
 0.2656846258+0.2960421110j |0100001000>

 0.3338905947+0.3206728741j |0100010100> +
 0.3238107199+0.1262004012j |0100101011> +
 0.0910500560+0.2228129615j |0101111100> +
 0.1630176264+0.2785495927j |0100001111> +
 0.3209220096+0.2481867521j |1010001010> +
 0.2665749112+0.3069556477j |1001101111> +
 0.1708213368+0.0106175848j |0000000011> +
 0.0339614876+0.2761098827j |0111100010> +
 0.0652667184+0.0370270346j |1100011100> +
 0.2363648831+0.0689371840j |1001110100>


In [7]:
psi = (psi_1 + psi_2).normalize
print(psi)
print()
print('Norm:', psi.conjugate * psi)

 0.1207889256+0.0075077662j |0000000011> +
 0.0591247650+0.2338632304j |0000001010> +
 0.0679882633+0.1231110324j |0001110010> +
 0.1680086861+0.2656201413j |0011111011> +
 0.1878674006+0.2093333842j |0100001000> +
 0.1152708691+0.1969643059j |0100001111> +
 0.2360963037+0.2267499638j |0100010100> +
 0.2289687559+0.0892371595j |0100101011> +
 0.2879754548+0.1547507961j |0101101011> +
 0.0643821120+0.1575525560j |0101111100> +
 0.0257629273+0.0217373566j |0110101000> +
 0.0240143982+0.1952391704j |0111100010> +
 0.2020876845+0.1724857181j |1001011011> +
 0.1884969274+0.2170504200j |1001101111> +
 0.1671352117+0.0487459503j |1001110100> +
 0.2269261292+0.1754945354j |1010001010> +
 0.0245936743+0.2071469913j |1010100011> +
 0.0461505392+0.0261820673j |1100011100> +
 0.1142292089+0.0595301025j |1100011101> +
 0.0252524530+0.0692003839j |1110101111>

Norm: (1+0j)


In [8]:
op = random_herm_op(N, 10)
print(op)
print()
print('Expectation value <psi|op|psi> =', psi.conjugate * op * psi)

1.5658972953+0.0000000000j YYZYYXIXIY +
0.4483677512+0.0000000000j ZYXXYIZYXX +
-9.1288490371+0.0000000000j XXZIXIIIZX +
0.3681644387+0.0000000000j XYXYXZZIZZ +
-2.6261201471+0.0000000000j XIXXZZYIXX +
-0.3807898893+0.0000000000j IIYYZXIYYY +
-2.7161775956+0.0000000000j XXZIYZIZZZ +
0.1095428346+0.0000000000j ZZYIZYXIIZ +
2.2303120538+0.0000000000j IIXZIZIIYI +
0.6386114868+0.0000000000j ZZIIXYZZXZ

Expectation value <psi|op|psi> = (0.025454063290106306+0j)


It is also possible to convert from a $2^N$ dimensional statevector over $N$ qubits to a `QuantumState` object, via the `array_to_QuantumState` function:

In [9]:
statevector = np.zeros(2**10)
statevector[0]   = np.sqrt(1/6)
statevector[250] = np.sqrt(2/6)
statevector[500] = np.sqrt(3/6)
psi = array_to_QuantumState(statevector)
print(psi)
print(psi.conjugate * psi)

 0.4082482905 |0000000000> +
 0.5773502692 |0011111010> +
 0.7071067812 |0111110100>
1.0


In [15]:
anz = random_herm_op(N, 5)
anz = AnsatzOp(anz.symp_matrix, anz.coeff_vec)
obs = random_herm_op(N, 10)
obs = ObservableOp(obs.symp_matrix, obs.coeff_vec)

In [17]:
print(anz)
print()
print(obs)

0.6240681842 YXXIIYXXII +
0.3185559876 IIIYYIXIIY +
1.0000000000 YYZIZXZZYX +
3.1391656069 ZYXIXXXYII +
1.6023890102 IIYIZIZZZX

-2.6863292736 IIIZYZZYIY +
-1.9272097317 ZXZXXIXIZX +
-9.2977828466 XXZXZXYZYX +
0.3143131581 ZYXYZIYIYX +
-0.7617950219 YIYIZYXYZZ +
-1.3126890715 ZYYXXXZZIZ +
3.1815403656 ZYXIXYYIZI +
-0.1075525226 IYYZXYZIYI +
-0.5188848850 ZYXYYIXYXY +
0.3722551847 ZXXYZZYXZI
